<a href="https://colab.research.google.com/github/Hijazzi/population-isochrone/blob/main/population_isochrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import modules, git

In [4]:
import openrouteservice as ors
import folium
import pandas as pd
import geopandas as gpd
import numpy as np
from turfpy.transformation import intersect
from turfpy.measurement import area
from geojson import Feature
from google.colab import files
from shapely.geometry import Point, Polygon

In [21]:
#Get dataset from git and merge
PATH_GEOJSON = 'https://raw.githubusercontent.com/dosm-malaysia/data-open/main/datasets/geodata/'
PATH_CENSUS = 'https://raw.githubusercontent.com/dosm-malaysia/data-open/main/datasets/census/'

use_vars = ['population_total','area_km2','nationality_citizen','nationality_non_citizen','sex_male', 'sex_female','ethnicity_proportion_bumi','ethnicity_proportion_chinese','ethnicity_proportion_indian','ethnicity_proportion_other','age_proportion_0_14','age_proportion_15_64','age_proportion_65_above','age_proportion_18_above']

df = pd.read_csv(PATH_CENSUS + 'census_parlimen.csv' ,usecols=['parlimen'] + use_vars) #filter columns
df['population_density'] = df['population_total']/df['area_km2']

df_merged = gpd.read_file(PATH_GEOJSON + 'electoral_0_parlimen.geojson')
df_merged = pd.merge(df_merged, df, on='parlimen', how='left')
df_merged['geometry2'] = df_merged.buffer(0)

df_merged.head()


,state,parlimen,code_state,code_parlimen,geometry,area_km2,population_total,nationality_citizen,nationality_non_citizen,sex_male,...,ethnicity_proportion_bumi,ethnicity_proportion_chinese,ethnicity_proportion_indian,ethnicity_proportion_other,age_proportion_0_14,age_proportion_15_64,age_proportion_65_above,age_proportion_18_above,population_density,geometry2
0,Perlis,P.001 Padang Besar,9,P.001,"MULTIPOLYGON (((100.20513 6.72227, 100.20778 6...",450,86798,84074,2724,44487,...,89.801841,5.640269,1.648548,2.909342,21.7,70.0,8.3,69.3,192.884444,"POLYGON ((100.20513 6.72227, 100.20778 6.71931..."
1,Perlis,P.002 Kangar,9,P.002,"MULTIPOLYGON (((100.16465 6.57050, 100.16618 6...",141,100755,98479,2276,49875,...,87.330294,9.576661,1.518090,1.574955,21.8,70.0,8.2,70.0,714.574468,"POLYGON ((100.16465 6.57050, 100.16618 6.56966..."
2,Perlis,P.003 Arau,9,P.003,"MULTIPOLYGON (((100.36581 6.48322, 100.36314 6...",222,97332,95902,1430,47983,...,89.417322,6.576505,2.338846,1.667327,17.9,75.0,7.1,74.8,438.432432,"POLYGON ((100.36581 6.48322, 100.36314 6.45419..."
3,Kedah,P.004 Langkawi,2,P.004,"MULTIPOLYGON (((99.79701 6.15851, 99.79812 6.1...",469,94138,90015,4123,48694,...,91.860246,5.122480,1.934122,1.083153,24.8,69.8,5.4,70.8,200.720682,"MULTIPOLYGON (((99.94513 6.31393, 99.94273 6.3..."
4,Kedah,P.005 Jerlun,2,P.005,"MULTIPOLYGON (((100.35617 6.44752, 100.35880 6...",316,66086,65407,679,32777,...,93.459416,4.178452,0.085618,2.276515,20.4,65.2,14.4,73.9,209.132911,"POLYGON ((100.35617 6.44752, 100.35880 6.44710..."


## Isochrones

In [18]:
client = ors.Client(key='KEY') #Enter your openrouteservice key here

x = 3.107263
y = 101.543541
coordinate = [[y, x]]
iso = client.isochrones(
    locations=coordinate,
    profile='driving-car',
    range_type = 'time',
    range=[300, 600, 900, 1200, 1500], #in second
    validate=False,
    attributes=['total_pop']
)

## Visualize map

In [23]:
color_scheme = ['#0d0887', '#7e03a8', '#cc4778', '#f89540', '#f0f921']

m = folium.Map(location=[x, y], zoom_start=10, tiles='openstreetmap')
#### isochrone layers
for i,isochrone in reversed(list(enumerate(iso['features']))):
  #if i==4:

    ###intersection layers to find average population
    total_population = 0
    for k in range(len(df_merged)):
      b = gpd.GeoSeries(df_merged['geometry2'][k]).__geo_interface__
      area_b = b['features'][0]['geometry']
      inter = intersect([isochrone['geometry'], area_b])
      if inter == None:
        pass
      else:
        population_density = df_merged['population_density'][k]
        area_intersection = area(inter)/1000000
        total_population += int(area_intersection*population_density)

    #map isochrone layers
    fillColor = color_scheme[i]
    color = 'black'
    geo_j = folium.GeoJson(data=isochrone,
                           style_function=lambda x, fillColor = fillColor, color = color: {"fillColor": fillColor,"color": color,},
                           #tooltip = str(str(int(isochrone['properties']['value']/60)) +' minutes')
                           tooltip = str('Estimated population: ' + str(total_population) +", " + str(int(isochrone['properties']['value']/60)) + ' minutes drivetime')
                           )
    geo_j.add_to(m)

folium.Marker(
      location=[x,y],
      popup="start here",
   ).add_to(m)
  
m